In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
import plotly.graph_objects as graph_objects
import findspark

findspark.init()

# spark_context = SparkContext.getOrCreate()
spark_context = SparkContext('local', 'R3_A3')
spark_session = SparkSession(spark_context)
sqlContext = SQLContext(spark_context)

/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [25]:
text_file = sqlContext.read.format('com.databricks.spark.csv').\
    options(header='true', inferschema='true', quote='"', delimiter=',').\
    load('./work/data/Covid19.csv')

rddfiltro = text_file.rdd.map(tuple)
rddGen = rddfiltro.filter(lambda word: word[3] in ['August'])
rddGen = rddGen.map(lambda word: (word[7], word[5] + word[6]))
# rddGen.take(10)

In [26]:
rddConteo = rddGen.reduceByKey(lambda a, b: a+b)
print(f'Conteo total -> {rddConteo.collect()}')

Conteo total -> [('Afghanistan', 1751), ('Argentina', 227333), ('Armenia', 5339), ('Aruba', 1884), ('Australia', 9789), ('Austria', 6224), ('Azerbaijan', 4839), ('Bahamas', 1688), ('Bahrain', 10862), ('Bangladesh', 77098), ('Barbados', 63), ('Belarus', 4145), ('Belgium', 15993), ('Belize', 957), ('Benin', 380), ('Bermuda', 13), ('Bhutan', 123), ('Bolivia', 42806), ('Bonaire, Saint Eustatius and Saba', 4), ('Bosnia_and_Herzegovina', 8609), ('Botswana', 833), ('Brazil', 1281774), ('British_Virgin_Islands', 29), ('Brunei_Darussalam', 3), ('Bulgaria', 5009), ('Burkina_Faso', 253), ('Burundi', 58), ('Cambodia', 40), ('Cameroon', 1911), ('Canada', 12343), ('Cape_Verde', 1496), ('Cayman_Islands', 2), ('Central_African_Republic', 97), ('Chad', 79), ('Chile', 58305), ('China', 2469), ('Colombia', 331472), ('Comoros', 45), ('Congo', 625), ('Costa_Rica', 22687), ('Cote_dIvoire', 1985), ('Croatia', 5092), ('Cuba', 1383), ('Curaçao', 28), ('Cyprus', 405), ('Czechia', 8069), ('Democratic_Republic_of

In [27]:
filtros = ['Guatemala']
rddOrden = spark_context.parallelize(
    rddConteo.filter(lambda a: a[0] in filtros).take(1))
print(f'Total de casos de COVID-19 en {filtros} -> {rddOrden.collect()}')

Total de casos de COVID-19 en ['Guatemala'] -> [('Guatemala', 25959)]


In [28]:
rddNombres = rddOrden.map(lambda x: (x[0]))
print(rddNombres.collect())

rddTotales = rddOrden.map(lambda x: (x[1]))
print(rddTotales.collect())

['Guatemala']
[25959]


In [29]:
graph = graph_objects.Figure(
    data=graph_objects.Pie(
        labels=rddNombres.collect(),
        values=rddTotales.collect()
    ))

graph.update_layout(
    title_text='Total de casos y muertes en Guatemala en el mes de Agosto',
    title_font_size=30)

graph.update_traces(
    hoverinfo='label+percent',
    textinfo='value',
    textfont_size=20)

graph.write_html('./work/reports/R3_A3.html', auto_open=True)